In [22]:
#csv로 저장된 파일 불러오기

import pandas as pd
df = pd.read_csv('finalFile.csv', header=0, encoding='cp949', engine='python')

In [23]:
#colums 확인

col = df.columns.tolist()
print(col)

['left', 'bottom', 'right', 'top', 'lon', 'lat', '학교', '도서관', '문화시설', '주민자치센터', '체육시설', '민원행정기관', 'cctv', '주차장']


In [24]:
#colums 중에 불법주정차 감소요인에 관련된 변수만 선정
score =  ['학교', '도서관', '문화시설', '주민자치센터', '체육시설', '민원행정기관', 'cctv', '주차장']
print(score)

#가중치변수를 score수 만큼 만들어줌.
weight = [1] * len(score)

['학교', '도서관', '문화시설', '주민자치센터', '체육시설', '민원행정기관', 'cctv', '주차장']


In [25]:
'''
각각의 변수에 가중치를 줌
학교 = 2
도서관 = 1
문화시설 = 1
주민자치센터 = 1
체육시설 = 1
민원행정기관 = 1
cctv = 2
주차장 = 3
'''
weight = [2,1,1,1,1,1,2,3] 

In [26]:
#null값 0으로 처리


df.fillna(0, inplace=True)


In [27]:
# 정규화 

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

df[score] = scaler.fit_transform(df.loc[:,score])
df.reset_index(inplace=True)

In [28]:
# 총점 계산

for i in range(0, len(score)):
    name = score[i]
    df[name] = df[name] * weight[i] 

df.set_index(['index','left','bottom','right','top','lon','lat'], inplace=True)

df['sum'] = df.sum(1)
df.reset_index(inplace=True)


In [29]:
# 등수 계산 
df['ranking'] = df['sum'].rank(method='dense', ascending= True).astype(int)

final = df.sort_values(by='ranking')
final.reset_index(drop=True, inplace=True)


In [30]:
# 상위 5개만 학인
display(final.head(5))

# 최종 결과 CSV 파일쓰기
# final.to_csv('결과_총점.csv', index=False, sep=',', encoding='cp949')


,index,left,bottom,right,top,lon,lat,학교,도서관,문화시설,주민자치센터,체육시설,민원행정기관,cctv,주차장,sum,ranking
0,0,937200,1953600,937500,1953900,126.790447,37.581006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,2089,959700,1950900,960000,1951200,127.045413,37.557927,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,2094,959700,1952400,960000,1952700,127.045331,37.571447,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,2117,959700,1965900,960000,1966200,127.044589,37.693125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,2118,959700,1966200,960000,1966500,127.044573,37.695829,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [19]:
#folium 을 활용해 지도에 상위 20위 공영주차장 최적입지를 확인함.

cnt = 20 

import folium

for i in range(cnt):
    map = folium.Map(location=[final.loc[i,'lat'],final.loc[i,'lon']], zoom_start = 15)
    folium.Marker([final.loc[i,'lat'],final.loc[i,'lon']]).add_to(map)
    print (final.loc[i,'lat'],final.loc[i,'lon'])
    j = i+1
    display(f'상위 {j}번째 격자의 센터 위치 보기', map)


보고자 하는 상위권 갯수는 5개 입니다.
37.58100598 126.79044740000002


'상위 1번째 격자의 센터 위치 보기'

37.55792683 127.04541329999999


'상위 2번째 격자의 센터 위치 보기'

37.57144676 127.0453311


'상위 3번째 격자의 센터 위치 보기'

37.69312468 127.0445892


'상위 4번째 격자의 센터 위치 보기'

37.6958286 127.04457260000001


'상위 5번째 격자의 센터 위치 보기'